# Bank Marketing Campaign Data
## A Datacamp Project
## Cleaning data with pandas
### by Hannah Bella C. Arceño

![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [154]:
import pandas as pd
import numpy as np

# Load bank_marketing.csv
df_bank_marketing = pd.read_csv('bank_marketing.csv')
print(">>> df_bank_marketing")
print(f'Shape: {df_bank_marketing.shape}')
print(f'Columns: {df_bank_marketing.columns}')

>>> df_bank_marketing
Shape: (41188, 16)
Columns: Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')


In [155]:
# Create client dataframe
client = df_bank_marketing[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

# REFACTOR: changing values to NaN for all columns

# Cleaning columns
# job: Replace "." into "_"
client['job'] = client['job'].str.replace(".", "_")
client.loc[client['job'].str.lower() == 'unknown', 'job'] = np.NaN

# marital
client.loc[client['marital'].str.lower() == 'unknown', 'marital'] = np.NaN

# education
client['education'] = client['education'].str.replace(".", "_")
client.loc[client['education'].str.lower() == 'unknown', 'education'] = np.NaN

# credit_default 
#! COULD BE BETTER...?
client.loc[client['credit_default'].str.lower() == 'unknown', 'credit_default'] = np.NaN
client.loc[client['credit_default'].str.lower() == 'yes', 'credit_default'] = True
client.loc[client['credit_default'].str.lower() == 'no', 'credit_default'] = False
client['credit_default'] = client['credit_default'].astype('bool')

# mortgage
client.loc[client['mortgage'].str.lower() == 'unknown', 'mortgage'] = np.NaN
client.loc[client['mortgage'].str.lower() == 'yes', 'mortgage'] = True
client.loc[client['mortgage'].str.lower() == 'no', 'mortgage'] = False
client['mortgage'] = client['mortgage'].astype('bool')

print(client.head())

# Show unique values per column
print("\n Unique values per column:")
for column in client.columns:
    print(f'{column}: {client[column].unique()}')

client.to_csv('client.csv', index=None)

   client_id  age        job  marital    education  credit_default  mortgage
0          0   56  housemaid  married     basic_4y           False     False
1          1   57   services  married  high_school            True     False
2          2   37   services  married  high_school           False      True
3          3   40     admin_  married     basic_6y           False     False
4          4   56   services  married  high_school           False     False

 Unique values per column:
client_id: [    0     1     2 ... 41185 41186 41187]
age: [56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44
 42 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67
 73 88 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91
 86 98 94 84 92 89]
job: ['housemaid' 'services' 'admin_' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' nan 'entrepreneur' 'student']
marital: ['married' 'single' 'divorced' nan]
education: ['basic_4

In [156]:
# Create campaign
campaign = df_bank_marketing[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'month', 'day']]

# Convert 'unknown' to np.naN
col_strings = []
for column in col_strings:
    campaign.loc[campaign[column].str.lower() == 'unknown', column] = np.NaN

# Convert yes/no to boolean values: True/False
col_boolean = ['previous_outcome', 'campaign_outcome']
for column in col_boolean:
    campaign.loc[campaign[column].str.lower() == 'yes', column] = True
    campaign.loc[campaign[column].str.lower() == 'no', column] = False
    campaign[column] = campaign[column].astype('bool')


# Create 'last_campaign_date' column
print(campaign['month'].unique())


month_to_num = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
                    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 
                    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

campaign['month'] = campaign['month'].replace(month_to_num)
# df_campaign['month'] = df_campaign['month'].astype('string')
# df_campaign['day'] = df_campaign['day'].astype('string')
campaign['last_contact_date'] = pd.to_datetime('2022' + "-" + campaign["month"].astype('string') + "-" + campaign["day"].astype('string'))

campaign.drop(columns=["month", "day"], inplace=True)

print(campaign.head())
campaign.to_csv('campaign.csv', index=None)

['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2022-05-13
1          1                1  ...             False         2022-05-19
2          2                1  ...             False         2022-05-23
3          3                1  ...             False         2022-05-27
4          4                1  ...             False         2022-05-03

[5 rows x 7 columns]


In [157]:
# Create economics.csv
economics = df_bank_marketing[['client_id', 'cons_price_idx', 'euribor_three_months']]
# print(df_economics['euribor_three_months'].dtype)

print(economics.head())
economics.to_csv('economics.csv', index=None)

   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857
